In [1]:
"""
This script fixes an anomalous value in the statistics of my water meter sensor.
It replaces the anomalous values by interpolated values and adjusted both the 
long term as well as the short term statistics tables

Before executing the script, stop you homeassistant service:
ssh jan@192.168.1.25
sudo systemctl stop home-assistant@homeassistant.service 

Don't forget to start afterwards:
sudo systemctl start home-assistant@homeassistant.service 
"""

"\nThis script fixes an anomalous value in the statistics of my water meter sensor.\nIt replaces the anomalous values by interpolated values and adjusted both the \nlong term as well as the short term statistics tables\n\nBefore executing the script, stop you homeassistant service:\nssh jan@192.168.1.25\nsudo systemctl stop home-assistant@homeassistant.service \n\nDon't forget to start afterwards:\nsudo systemctl start home-assistant@homeassistant.service \n"

In [2]:
hass_ip="192.168.1.25" # change this to the ip of you home assistant installation

# Retrieve database from home assistant server

In [3]:
! scp homeassistant@{hass_ip}:/home/homeassistant/.homeassistant/home-assistant_v2.db .
! mkdir -p backup
! cp home-assistant_v2.db backup # make a backup

home-assistant_v2.db                          100%  590MB  10.0MB/s   00:59    


# Connect to DB

In [4]:
import pandas
import sqlite3
import pandas
from IPython.display import display

pandas.options.plotting.backend = "plotly"

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("home-assistant_v2.db")

# Find watermeter

In [5]:
df=pandas.read_sql_query(f"select * FROM statistics_meta --where unit_of_measurement='EUR';", con)
df.head()

,id,statistic_id,source,unit_of_measurement,has_mean,has_sum,name
0,1,sensor.nibe_93714_40067,recorder,°C,1,0,None
1,2,sensor.nibe_93714_40014,recorder,°C,1,0,None
2,3,sensor.nibe_93714_40013,recorder,°C,1,0,None
3,4,sensor.nibe_93714_40004,recorder,°C,1,0,None
4,5,sensor.nibe_93714_40083,recorder,A,1,0,None


In [6]:
df[df['statistic_id'].str.contains('water')]

,id,statistic_id,source,unit_of_measurement,has_mean,has_sum,name
80,81,sensor.watermeter_value_from_json,recorder,m³,0,1,None
81,82,sensor.watermeter_value_from_json_cost,recorder,EUR,0,1,None


# Find anomaly

In [7]:
sensor_id=81
# sensor_id=82

df=pandas.read_sql_query(f"select * FROM statistics where metadata_id={sensor_id};", con)
display(df.tail())
df=df.set_index('start')
fig=df[['state','sum']].plot()
fig.show()

df=pandas.read_sql_query(f"select * FROM statistics_short_term where metadata_id={sensor_id};", con)
display(df.tail())
df=df.set_index('start')
fig=df[['state','sum']].plot()
fig.show()

# Fix anomaly

In [9]:
for sensor_id in [81,82]:
    threshold = 120  # derived from the graphs, coincidence that it applies for both sensors

    for table in ["statistics", "statistics_short_term"]:
        # read what is currently in the db
        df = pandas.read_sql_query(
            f"select * FROM {table} where metadata_id = {sensor_id};", con
        )
        # set anomalous value to NA
        df.loc[df["state"] > threshold, "state"] = pandas.NA
        # fill NA by interpolating
        df["state"] = df["state"].interpolate(method="linear").fillna(0)
        if sensor_id == 81:
            df["sum"] = df["state"] - 100.058
        elif sensor_id == 82:
            if table == "statistics":
                df["sum"] = df["state"].diff(periods=1).apply(lambda x: max(x,0)).fillna(0).cumsum()
            elif table == "statistics_short_term":
                df["sum"] = df["state"].diff(periods=1).apply(lambda x: max(x,0)).fillna(0).cumsum() # missing the offset
                # calculating the offset by finding the first common date between
                # short term statistics and long term statistics
                df_long = pandas.read_sql_query(
                    f"select * FROM statistics where metadata_id = {sensor_id};", con
                )
                common_dt = df.loc[pandas.to_datetime(df["start"]).dt.minute == 0, "start"].min()
                offset = (
                    df_long.loc[df_long["start"] == common_dt, "sum"].iloc[0]
                    - df.loc[df["start"] == common_dt, "sum"].iloc[0]
                )
                df["sum"]  = df["sum"] + offset

        # drop all existing rows with same metadata_id from the sensor
        # because i will recreate all statistics for that sensor
        # instead of updating the sum value which one would do if you would like
        # to keep the existing statistics

        stmnt = f"""DELETE FROM {table}
                    WHERE metadata_id={sensor_id};"""
        cur = con.cursor()
        cur.execute(stmnt)
        con.commit()

        # insert new data into table
        df.to_sql(
            table, con, schema=None, if_exists="append", index=False,
        )

# Check if fixed

In [12]:
sensor_id=81
# sensor_id=82

df=pandas.read_sql_query(f"select * FROM statistics where metadata_id={sensor_id};", con)
display(df.tail())
df=df.set_index('start')
fig=df[['state','sum']].plot()
fig.show()

df=pandas.read_sql_query(f"select * FROM statistics_short_term where metadata_id={sensor_id};", con)
display(df.tail())
df=df.set_index('start')
fig=df[['state','sum']].plot()
fig.show()

,id,created,start,mean,min,max,last_reset,state,sum,metadata_id
921,433576,2023-02-25 18:00:10.685034,2023-02-25 17:00:00.000000,None,None,None,None,107.148,7.090,81
922,433683,2023-02-25 19:00:10.676403,2023-02-25 18:00:00.000000,None,None,None,None,107.148,7.090,81
923,433790,2023-02-25 20:00:10.691024,2023-02-25 19:00:00.000000,None,None,None,None,107.148,7.090,81
924,433897,2023-02-25 21:00:10.674598,2023-02-25 20:00:00.000000,None,None,None,None,107.148,7.090,81
925,434004,2023-02-25 22:00:10.704659,2023-02-25 21:00:00.000000,None,None,None,None,107.149,7.091,81


,id,created,start,mean,min,max,last_reset,state,sum,metadata_id
5436,4259656,2023-02-25 21:40:10.663924,2023-02-25 21:35:00.000000,None,None,None,None,107.148,7.090,81
5437,4259763,2023-02-25 21:45:10.671089,2023-02-25 21:40:00.000000,None,None,None,None,107.148,7.090,81
5438,4259870,2023-02-25 21:50:10.633405,2023-02-25 21:45:00.000000,None,None,None,None,107.148,7.090,81
5439,4259977,2023-02-25 21:55:10.635846,2023-02-25 21:50:00.000000,None,None,None,None,107.149,7.091,81
5440,4260084,2023-02-25 22:00:10.659872,2023-02-25 21:55:00.000000,None,None,None,None,107.149,7.091,81


# Copy db back to home assistant server

In [10]:
! scp home-assistant_v2.db homeassistant@{hass_ip}:/home/homeassistant/.homeassistant/

home-assistant_v2.db                          100%  590MB   9.2MB/s   01:04    
